## Snowflake Data Import  
Import data from GCS to a previously created BigQuery dataset.  

This Notebook assumes that you've already generated data at one or more scale factors and uploaded them to the project Google Cloud Storage bucket listed in `config.gcs_data_bucket`  

Three values are required to initiate an upload to BigQuery:  
1. `test` - the test name, either `h` or `ds`
2. `scale` - the scale factor in GB, usually this will be `1, 100, 1000, 10000`  
3. `name` - name of this instance of the `test` and `scale` combination, i.e. `time-partitioned`

In [ ]:
import sf_tpc, config

In [ ]:
test = ""
scale = 1
cid = "01"

In [ ]:
sf = sf_tpc.SFTPC(test=test,
                  scale=scale,
                  cid=cid,
                  warehouse="TEST9000_XSMALL",
                  desc="data_upload",
                  verbose=True,
                  verbose_query=True)

In [ ]:
print('Using database:', sf.database)

In [ ]:
sf._connect()

In [ ]:
sf.role("SYSADMIN")

In [ ]:
sf.database_create()

In [ ]:
sf.database_use()

In [ ]:
sf.create_schema({"ds": config.fp_sf_ds_schema,
                  "h": config.fp_sf_h_schema}[test],
                 verbose=False)

In [ ]:
sf.gcs_integration_drop()
sf.gcs_integration_create()

In [ ]:
sf.grant_storage_integration_access()

In [ ]:
sf.gcs_inventory()

In [ ]:
sf.create_named_file_format()

In [ ]:
sf.warehouse_use()

In [ ]:
tables = sf.df_gcs.table.unique()
for table in tables: print(table, "  ")

In [ ]:
%%time
sf.import_table("reason")

In [ ]:
sf.fp_log

In [ ]:
df = sf_tpc.parse_log(sf.fp_log)
df